## Filter read alignments in bam file based on percent id with md value indicating percent id...

In [1]:
import pysam

In [1]:
test_md = '103^T53C2T341'

def _match_len(md):
    length=0
    mismatch=0
    number=""
    for i, c in enumerate(md):
        try:
            val = int(c)
            number = number+c
        except:
            if len(number) > 0:
                length += int(number)
                number=""
        if i == len(md)-1:
            length += int(number)
    return length

assert _match_len(test_md) == 499

In [3]:


md = lambda b: str(b).split("\t")[-1].split(",")[3].replace(")","").replace("'","").strip()

def filter_bam(bam, outbam, pctid = 95):
    with pysam.AlignmentFile(bam, "rb") as ih, pysam.AlignmentFile(outbam, "wb", template=ih) as oh:
        good = 0
        total = 0
        for i, l in enumerate(ih):
            if l.is_duplicate:
                continue

            total += 1
            md = str(l).split("\t")[-1].split(",")[3].replace(")","").replace("'","").strip()  #get md value from raw bam entry
            match = match_len(md)
            pct_match = (match)/l.rlen * 100

            if pct_match > pctid:
                good += 1
                oh.write(l)
        print("there were %s good read alignments out of %s total alignments" % (good, total))

In [2]:
bam = "../data/big_data/bam_exe/SRS019030_454_vs_OlaneusYIT12061.bam"

In [21]:
with pysam.AlignmentFile(bam, "rb") as ih: #, pysam.AlignmentFile(outbam, "wb", template=ih) as oh:
    good = 0
    total = 0
    for i, l in enumerate(ih):     
        print(l.cigartuples)
        if l.query_length != l.infer_query_length() or l.query_length != len(l.query_sequence):
            print(l.query_length, l.infer_query_length(), len(l.query_sequence))
        if i == 100:
            break

[(4, 42), (0, 57), (4, 178)]
[(4, 25), (0, 57), (4, 377)]
[(4, 310), (0, 57), (4, 65)]
[(4, 21), (0, 57), (4, 402)]
[(4, 21), (0, 57), (4, 384)]
[(4, 47), (0, 57), (4, 271)]
[(4, 93), (0, 57), (4, 216)]
[(4, 2), (0, 57), (4, 387)]
[(4, 112), (0, 57), (4, 326)]
[(4, 191), (0, 57), (4, 280)]
[(4, 329), (0, 57), (4, 113)]
[(4, 48), (0, 57), (4, 413)]
[(4, 269), (0, 57), (4, 100)]
[(4, 375), (0, 57), (4, 90)]
[(4, 117), (0, 57), (4, 305)]
[(4, 25), (0, 57), (4, 289)]
[(4, 23), (0, 57), (4, 370)]
[(4, 50), (0, 57), (4, 398)]
[(4, 404), (0, 58)]
[(4, 48), (0, 57), (4, 399)]
[(4, 424), (0, 57), (4, 26)]
[(4, 472), (0, 58)]
[(4, 368), (0, 57), (4, 91)]
[(4, 357), (0, 57), (4, 90)]
[(4, 18), (0, 57), (4, 258)]
[(4, 126), (0, 57), (4, 306)]
[(4, 87), (0, 57), (4, 373)]
[(4, 356), (0, 57), (4, 111)]
[(4, 211), (0, 50), (4, 213)]
[(4, 56), (0, 50), (4, 279)]
[(4, 92), (0, 50), (4, 273)]
[(4, 207), (0, 45), (4, 133)]
[(4, 366), (0, 40), (4, 95)]
[(4, 234), (0, 40), (4, 247)]
[(4, 314), (0, 40), (4,